# Как можно улучшить классификацию тактик без значимых слов

- можно априор сделать равномерным, но тогда может увеличиться ошибочная классификация пустых предложений
- возможно добавить n gramms, которые хорошо находят эти тактики

# TO DO

- проверить на дубли датасет, разобраться с мусором
data[data['sentence'].duplicated(keep=False)].sort_values(['sentence'])
    - вообoе сделать ноутбук для обработки последующих текстов
- нужна отдельная обработка проекта по mittre - на его основе 
    - изучение основных разделителей атак
    - изучение схожих описаний эмбеддингов
    - тянуть url откуда предложения появились в датасет
      
- оценить какие тактики сложно спрогнозировать и почему
    - для exfiltration, impact практически нет значимых слов, а для defense-evasion есть такие
    - у некоторых классов одинаковые значимые слова могут быть - impact exfiltration

- определить, каких тактик прям мало в выборках, из-за этого и ошибки
- min_df=10, max_df=0.1 для tf/idf на уровень слов
